In [ ]:
"""
The Autonomous Cooperative Consensus Orbit Determination (ACCORD) framework.
Author: Beth Probert
Email: beth.probert@strath.ac.uk

Copyright (C) 2025 Applied Space Technology Laboratory

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

"""

import json
import numpy as np
from datetime import datetime, timedelta
from src.consensus_mech import ConsensusMechanism
from src.dag import DAG
from src.satellite_node import SatelliteNode
from src.transaction import Transaction, TransactionMetadata
from src.utils import load_json_data


# Tester demo function
def run_consensus_test():
    # Number of transactions must be >= 3 for consensus to be possible
    n_tx = 3
    print("=== Running Consensus Test ===")

    # Load observation data from your JSON file
    satellites = load_json_data("od_data.json")
    if not satellites:
        print("No satellite data found.")
        return

    # Setup
    test_satellite = SatelliteNode(node_id="SAT-001")
    test_dag = DAG()
    base_sat = satellites[0]

    # Add other transactions with some random noise
    # TODo - test with bigger variants
    for i in range(n_tx):
        # Simulate noise in observation
        noise = {
            "mean_motion": np.random.normal(0, 0.01),       # rev/day
            "eccentricity": np.random.normal(0, 0.00002),    # unitless
            "inclination": np.random.normal(0, 0.05),        # degrees
            "epoch_jitter": np.random.normal(0, 2)           # seconds
        }

        observed_epoch = base_sat.epoch.utc_datetime() + timedelta(seconds=noise["epoch_jitter"])

        dummy_tx = Transaction(
            sender_address=f"NODE-{i}",
            recipient_address="GROUND",
            sender_private_key="dummy_key",
            metadata=TransactionMetadata(timestamp=datetime.now()),
            tx_data=json.dumps({
                "OBJECT_NAME": base_sat.name,
                "OBJECT_ID": 25544,
                "EPOCH": observed_epoch.isoformat() + "Z",
                "MEAN_MOTION": ((base_sat.model.no_kozai / (2 * np.pi)) * 1440) + noise["mean_motion"],
                "ECCENTRICITY": base_sat.model.ecco + noise["eccentricity"],
                "INCLINATION": (base_sat.model.inclo * 180 / np.pi) + noise["inclination"]
            })
        )
        test_dag.add_tx(dummy_tx)

    test_transaction = Transaction(sender_address=test_satellite.id,
                                   recipient_address="GROUND",
                                   sender_private_key="dummy_key",
                                   metadata=TransactionMetadata(timestamp=datetime.now()),
                                   tx_data="")

    # Init PoISE mechanism
    poise = ConsensusMechanism()

    # Run consensus on a single satellite observation
    consensus_result = poise.proof_of_inter_satellite_evaluation(
        dag=test_dag,
        sat_node=test_satellite,
        transaction=test_transaction
    )

    # Output result
    print(f"\nConsensus reached: {consensus_result}")
    print(f"Updated Node Reputation: {test_satellite.reputation:.2f}")
    print(f"Transaction:\n{test_transaction}")
    last_index = list(test_dag.ledger)[-1]
    penultimate_index = list(test_dag.ledger)[-2]
    print("Final Tx:")
    print(test_dag.ledger[last_index])
    print("Penultimate Tx:")
    print(test_dag.ledger[penultimate_index])

run_consensus_test()


=== Running Consensus Test ===

Consensus reached: True
Updated Node Reputation: 0.10
Transaction:
Transaction(
  sender_address=SAT-001,
  recipient_address=GROUND,
  timestamp=2025-07-29 11:43:51.935555,
  tx_data='{"OBJECT_NAME": "ISS (ZARYA)", "OBJECT_ID": 25544, "EPOCH": "2024-05-06T19:53:05Z", "MEAN_MOTION": 15.509576740000002, "ECCENTRICITY": 0.000358, "INCLINATION": 51.639300000000006}',
  consensus_reached=True,
  is_confirmed=False,
  is_rejected=False,
  confirmation_score=0.1,
  parent_hashes=['abeff4df30bbe5c0e02d246a97c59c7f2437542b63ca7ac575ce692c3a0868a7', 'Genesis Transaction 1'],
  hash=36095a9009...)

Final Tx:
[Transaction(
  sender_address=SAT-001,
  recipient_address=GROUND,
  timestamp=2025-07-29 11:43:51.935555,
  tx_data='{"OBJECT_NAME": "ISS (ZARYA)", "OBJECT_ID": 25544, "EPOCH": "2024-05-06T19:53:05Z", "MEAN_MOTION": 15.509576740000002, "ECCENTRICITY": 0.000358, "INCLINATION": 51.639300000000006}',
  consensus_reached=True,
  is_confirmed=False,
  is_rejected

=== Running Consensus Test ===

Consensus reached: True
Updated Node Reputation: 0.10
Transaction:
Transaction(
  sender_address=SAT-001,
  recipient_address=GROUND,
  timestamp=2025-07-28 17:34:13.968365,
  tx_data='{"OBJECT_NAME": "ISS (ZARYA)", "OBJECT_ID": 25544, "EPOCH": "2024-05-06T19:53:05Z", "MEAN_MOTION": 15.509576740000002, "ECCENTRICITY": 0.000358, "INCLINATION": 51.639300000000006}',
  consensus_reached=True,
  is_confirmed=False,
  is_rejected=False,
  confirmation_score=0.1,
  parent_hashes=['6f62136e20c8c475c1c6478579d179204450a55bca4a219a636860209570419c', 'Genesis Transaction 2'],
  hash=53d3302c73...)

Final Tx:
[Transaction(
  sender_address=SAT-001,
  recipient_address=GROUND,
  timestamp=2025-07-28 17:34:13.968365,
  tx_data='{"OBJECT_NAME": "ISS (ZARYA)", "OBJECT_ID": 25544, "EPOCH": "2024-05-06T19:53:05Z", "MEAN_MOTION": 15.509576740000002, "ECCENTRICITY": 0.000358, "INCLINATION": 51.639300000000006}',
  consensus_reached=True,
  is_confirmed=False,
  is_rejected